# GeoAPI price comparison

Last updated 2019-06-19

The applet at the bottom of the notebook estimates the monthly charge for using various GeoAPI providers in your project.

In [1]:
import numpy as np
from ipywidgets import Layout, Button, Box
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

https://cloud.google.com/maps-platform/pricing/sheet/

In [2]:
def google(geocode=0, route=0, distmat=0, freecredit=True):
    funclist = [
        lambda x: (x * 0.004) + 500,
        lambda x: x * 0.005,
        lambda _: np.nan
    ]

    credit = 0.
    if freecredit:
        credit = 200.
    out = geocode + route + distmat
    condlist = [
        (100000 < out) and (out <= 500000),
        (0 <= out) and (out <= 100000),
    ]
    out = np.piecewise(geocode + route + distmat,
                       condlist=condlist,
                       funclist=funclist) - credit
    return np.piecewise(out, condlist=[out > 0],
                        funclist=[
                            lambda x: x,
                            0
                        ])

assert google() == 0

https://developers.arcgis.com/pricing/credits/

In [3]:
def esri(geocode=0, route=0, distmat=0, freecredit=True):
    nans = geocode + route + distmat > 1000000

    geocode = geocode * 0.004
    route = route * 0.0005
    distmat = distmat * 0.00005
    credit = 0.
    if freecredit:
        credit = 5.

    out = (geocode + route + distmat) - credit

    if type(out) is np.ndarray:
        out[nans] = np.nan
    elif nans:
        return np.nan

    return np.piecewise(out, condlist=[out > 0],
                        funclist=[
                            lambda x: x,
                            0.
                        ])

assert esri() == 0

https://geocoder.ca/pricing

In [4]:
def geocoder_ca(geocode=0, route=0, distmat=0, freecredit=True):
    assert np.sum(route) == 0
    assert np.sum(distmat) == 0

    if freecredit:
        geocode = geocode - 76050
    return np.piecewise(geocode,
                        condlist=[
                            geocode > 200000,
                            (geocode <= 200000) and (geocode >= 0)
                        ],
                        funclist=[
                            500.,
                            lambda x: x * 0.0025,
                            0.
                        ])

assert geocoder_ca() == 0

https://developer.here.com/plans

In [5]:
def here(geocode=0, route=0, distmat=0, freecredit=False):
    """
    for matrix Routing one transaction is counted for each start point request
    """
    out = geocode + route + distmat
    out = np.array(out, dtype=float)
    return np.piecewise(out,
                        condlist=[
                            (out > 449000) and (out <= 1000000),
                            (out > 250000) and (out <= 449000),
                            (out >= 0) and (out <= 250000)
                        ],
                        funclist=[
                            449.,
                            lambda x: x / 1000,
                            0.,
                            np.nan
                        ])

assert here() == 0

https://azure.microsoft.com/en-ca/pricing/details/azure-maps/

In [6]:
def azure(geocode=0, route=0, distmat=0, freecredit=True):
    price = 0.0005
    free_trans = 0
    if freecredit:
        free_trans = 25000
    if np.sum(distmat) > 0:
        price = 0.005
        free_trans = 0
    out = geocode + route + distmat
    out = np.array(out, dtype=float)
    return np.piecewise(out,
                        condlist=[out <= free_trans],
                        funclist=[
                            0.,
                            lambda x: (x - free_trans) * price
                        ])

assert azure() == 0

https://www.loqate.com/en-us/pricing/price-per-lookup/

In [7]:
def loqate(geocode=0, route=0, distmat=0, freecredit=False):
    """
    The requests are purchased in packages of $100, $300, or $1000
    and each package is good for 12 months

    so calculations are based on the average monthly cost over a year
    """
    assert np.sum(distmat) == 0
    geocode = geocode * 12
    geocode = np.piecewise(geocode,
                           condlist=[
                               geocode > 28669,
                               (geocode > 8012) and (geocode <= 28669),
                               (geocode > 2500) and (geocode <= 8012),
                               (geocode > 0) and (geocode <= 2500)
                           ],
                           funclist=[
                               lambda x: np.ceil(x / 28669) * 1000,
                               1000.,
                               300.,
                               100.,
                               0
                           ]) / 12.
    route = route * 12
    route = np.piecewise(route,
                         condlist=[
                             route > 14084,
                             (route > 3370) and (route <= 14084),
                             (route > 1000) and (route <= 3370),
                             (route > 0) and (route <= 1000)
                         ],
                         funclist=[
                             lambda x: np.ceil(x / 14084) * 1000,
                             1000.,
                             300.,
                             100.,
                             0
                         ]) / 12.
    return np.array(geocode + route, dtype=float)

assert loqate() == 0

https://www.geocod.io/pricing/

In [8]:
def geocodio(geocode=0, route=0, distmat=0, freecredit=True):
    assert np.sum(route) == 0
    assert np.sum(distmat) == 0
    geocode = np.array(geocode, dtype=float)
    if freecredit:
        geocode = geocode - 76050.
    return np.piecewise(geocode,
                        condlist=[
                            geocode > 1500000,
                            (geocode <= 1500000) and (geocode > 0)
                        ],
                        funclist=[
                            750.,
                            lambda x: x * 0.0005,
                            0.
                        ])

assert geocodio() == 0

In [9]:
geocoders = ["Google", "ESRI", "geocoder.ca", "Here",
             "Azure", "Loqate", "geocodio"]

routers = ["Google", "ESRI", "", "Here",
             "Azure", "Loqate", ""]

distmaters = ["Google", "ESRI", "", "Here",
             "Azure", "", ""]

In [10]:
def cost(geocoder:str, geocode:int,
         router:str, route:int,
         distmater:str, distmat:int):
    cost_functions = [google, esri, geocoder_ca, here, azure, loqate, geocodio]
    
    geocode_input = [0] * len(cost_functions)
    route_input = [0] * len(cost_functions)
    distmat_input = [0] * len(cost_functions)
    
    if geocoder in geocoders:
        geocode_input[geocoders.index(geocoder)] = geocode
        
    if router in routers:
        route_input[routers.index(router)] = route
        
    if distmater in distmaters:
        distmat_input[distmaters.index(distmater)] = distmat
        
    inputs = zip(geocode_input, route_input, distmat_input)
    out = {}
    
    for func, inp, name in zip(cost_functions, inputs, geocoders):
        try:
            funcout = func(*inp)
            if np.isnan(funcout):
                raise ValueError
            out[name] = float(funcout)
        except ValueError:
            out[name] = "Please contact " + name
    return out

def cost_to_html(*args, **kwargs):
    costs: dict = cost(*args, **kwargs)
    for service in costs.keys():
        if type(costs[service]) is float:
            costs[service] = "${:.2f}".format(costs[service])
    costs: pd.DataFrame = pd.DataFrame(zip(costs.keys(),costs.values()), columns=["Provider", "Monthly Charge (USD)"])
    return display(HTML(costs.to_html(index=False)))

In [11]:
selector_layout = Layout(display='flex',align_items='stretch', width='30%')
num_layout = Layout(display='flex',align_items='stretch', width='70%')
style = {'description_width': 'initial'}
layout=Layout(display='flex',width='95%')

geocoder_wig = widgets.Dropdown(
    options=geocoders,
    value='Google',
    description='Geocode provider',
    layout=layout,
    style=style
    
)

geocode_wig = widgets.IntSlider(
    value=1000,
    min=0,
    max=2500000,
    step=100,
    description='Geocodes per month',
    continuous_update=False,
    layout=layout,
    style=style
)

router_wig = widgets.Dropdown(
    options=[r for r in routers if r != ""],
    value='Google',
    description='Route provider',
    layout=layout,
    style=style
)

route_wig = widgets.IntSlider(
    value=1000,
    min=0,
    max=1000000,
    step=100,
    description='Routes per month',
    continuous_update=False,
    layout=layout,
    style=style
)

distmater_wig = widgets.Dropdown(
    options=[r for r in distmaters if r != ""],
    value='Google',
    description='Distance matrix provider',
    layout=layout,
    style=style
)

distmate_wig = widgets.IntSlider(
    value=1000,
    min=0,
    max=1000000,
    step=100,
    description='OD pairs per month',
    continuous_update=False,
    layout=layout,
    style=style
)

selector_wigs = widgets.VBox([geocoder_wig, router_wig, distmater_wig], layout=selector_layout)
num_wigs = widgets.VBox([geocode_wig, route_wig, distmate_wig], layout=num_layout)

input_wigs = widgets.HBox([selector_wigs, num_wigs], layout=Layout(display='flex',align_items='stretch', width='100%'))

output_wig = widgets.interactive_output(cost_to_html, dict(
                 geocoder=geocoder_wig, geocode=geocode_wig,
                 router=router_wig, route=route_wig,
                 distmater=distmater_wig, distmat=distmate_wig))

out = widgets.VBox([input_wigs, output_wig])

In [12]:
display(out)